In [34]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt
from joblib import dump
from sklearn.model_selection import train_test_split
import sklearn.naive_bayes

from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

%matplotlib inline

data_path = "./udemy_courses.csv"
df = pd.read_csv(data_path)

df

,course_id,course_title,url,is_paid,price,num_subscribers,num_reviews,num_lectures,level,content_duration,published_timestamp,subject
0,1070968,Ultimate Investment Banking Course,https://www.udemy.com/ultimate-investment-bank...,True,200,2147,23,51,All Levels,1.5 hours,2017-01-18T20:58:58Z,Business Finance
1,1113822,Complete GST Course & Certification - Grow You...,https://www.udemy.com/goods-and-services-tax/,True,75,2792,923,274,All Levels,39 hours,2017-03-09T16:34:20Z,Business Finance
2,1006314,Financial Modeling for Business Analysts and C...,https://www.udemy.com/financial-modeling-for-b...,True,45,2174,74,51,Intermediate Level,2.5 hours,2016-12-19T19:26:30Z,Business Finance
3,1210588,Beginner to Pro - Financial Analysis in Excel ...,https://www.udemy.com/complete-excel-finance-c...,True,95,2451,11,36,All Levels,3 hours,2017-05-30T20:07:24Z,Business Finance
4,1011058,How To Maximize Your Profits Trading Options,https://www.udemy.com/how-to-maximize-your-pro...,True,200,1276,45,26,Intermediate Level,2 hours,2016-12-13T14:57:18Z,Business Finance
...,...,...,...,...,...,...,...,...,...,...,...,...
3678,775618,Learn jQuery from Scratch - Master of JavaScri...,https://www.udemy.com/easy-jquery-for-beginner...,True,100,1040,14,21,All Levels,2 hours,2016-06-14T17:36:46Z,Web Development
3679,1088178,How To Design A WordPress Website With No Codi...,https://www.udemy.com/how-to-make-a-wordpress-...,True,25,306,3,42,Beginner Level,3.5 hours,2017-03-10T22:24:30Z,Web Development
3680,635248,Learn and Build using Polymer,https://www.udemy.com/learn-and-build-using-po...,True,40,513,169,48,All Levels,3.5 hours,2015-12-30T16:41:42Z,Web Development
3681,905096,CSS Animations: Create Amazing Effects on Your...,https://www.udemy.com/css-animations-create-am...,True,50,300,31,38,All Levels,3 hours,2016-08-11T19:06:15Z,Web Development


In [35]:
# Removendo-as
df = df.drop_duplicates(subset=['course_id'], keep='last')
df[df.duplicated(['course_id'])]

df[~df['url'].str.contains('https://www.udemy.com')]

df.loc[2066, 'url'] = "https://www.udemy.com/learnguitartoworship/"
df.loc[2066, 'is_paid'] = 'True' # aqui deixamos em string devido ao tratamento do próximo bloco.
df.loc[2066, 'price'] = 349
df.loc[[2066]]

c:\users\mnsilva\appdata\local\programs\python\python39\lib\site-packages\pandas\core\indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


,course_id,course_title,url,is_paid,price,num_subscribers,num_reviews,num_lectures,level,content_duration,published_timestamp,subject
2066,96698,Learn Classical Guitar Technique and play 'Spa...,https://www.udemy.com/learnguitartoworship/,True,349,30,1463,29,52,Beginner Level,3 hours,Musical Instruments


In [36]:
# Criando nossa coluna de slug
df['slug'] = df['url'].str.replace('https://www.udemy.com/', '').str.replace('/', '')
df.head(5)

<ipython-input-36-b980f0f6c88a>:2: FutureWarning: The default value of regex will change from True to False in a future version.
  df['slug'] = df['url'].str.replace('https://www.udemy.com/', '').str.replace('/', '')
<ipython-input-36-b980f0f6c88a>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['slug'] = df['url'].str.replace('https://www.udemy.com/', '').str.replace('/', '')


,course_id,course_title,url,is_paid,price,num_subscribers,num_reviews,num_lectures,level,content_duration,published_timestamp,subject,slug
0,1070968,Ultimate Investment Banking Course,https://www.udemy.com/ultimate-investment-bank...,True,200,2147,23,51,All Levels,1.5 hours,2017-01-18T20:58:58Z,Business Finance,ultimate-investment-banking-course
1,1113822,Complete GST Course & Certification - Grow You...,https://www.udemy.com/goods-and-services-tax/,True,75,2792,923,274,All Levels,39 hours,2017-03-09T16:34:20Z,Business Finance,goods-and-services-tax
2,1006314,Financial Modeling for Business Analysts and C...,https://www.udemy.com/financial-modeling-for-b...,True,45,2174,74,51,Intermediate Level,2.5 hours,2016-12-19T19:26:30Z,Business Finance,financial-modeling-for-business-analysts-and-c...
3,1210588,Beginner to Pro - Financial Analysis in Excel ...,https://www.udemy.com/complete-excel-finance-c...,True,95,2451,11,36,All Levels,3 hours,2017-05-30T20:07:24Z,Business Finance,complete-excel-finance-course-from-beginner-to...
4,1011058,How To Maximize Your Profits Trading Options,https://www.udemy.com/how-to-maximize-your-pro...,True,200,1276,45,26,Intermediate Level,2 hours,2016-12-13T14:57:18Z,Business Finance,how-to-maximize-your-profits-trading-options


In [37]:
# Variáveis necessárias
textos_slug = list(df['slug'])
textos_title = list(df['course_title'])
textos_slug_title = list(df['slug'] + ' ' + df['course_title'])

vocabulario_slug = []
vocabulario_title = []
vocabulario_slug_title = []

# Separando nosso vocabulário de Slug
# E alimentamos nossa primeira parte do vocubulário que compõem slugs e títulos

for text in textos_slug:
    palavras = text.lower().split('-')
    for t in palavras:
        if t not in vocabulario_slug:
            vocabulario_slug.append(t)

vocabulario_slug_title = vocabulario_slug.copy()

# Criando nosso vocabulário de Títulos
# E alimentando nossa última parte do vocabulário de slugs e títulos (sem repetições)

for text in textos_title:
    palavras = text.lower().split(' ')
    for t in palavras:
        if t not in vocabulario_title:
            vocabulario_title.append(t)

        if t not in vocabulario_slug_title:
            vocabulario_slug_title.append(t)



print('vocabulario_slug \t\t' + str(len(vocabulario_slug)) + ' palavras.')
print('vocabulario_title \t\t' + str(len(vocabulario_title)) + ' palavras.')
print('vocabulario_slug_title \t\t' + str(len(vocabulario_slug_title)) + ' palavras.')

vocabulario_slug 		3578 palavras.
vocabulario_title 		4644 palavras.
vocabulario_slug_title 		5595 palavras.


In [38]:
dump(vocabulario_slug_title, 'vocabulario_slug_title.joblib')


['vocabulario_slug_title.joblib']

In [39]:
# Variáveis necessárias

amostra_slug_title = []



# Amostras dos Títulos + Slugs (sem repetições)

for t in textos_slug_title:
    amostra = []
    for p in vocabulario_slug_title:
        if p in t.lower().replace('-', ' ').split(' '):
            amostra.append(1)
        else:
            amostra.append(0)

    amostra_slug_title.append(amostra)


ok


In [41]:
# Obtendo os arrays das nossas amostras

amostra_slug_title = np.array(amostra_slug_title)

from sklearn.naive_bayes import BernoulliNB, ComplementNB, MultinomialNB
from sklearn.metrics import accuracy_score

# Modelo a ser utilizado de Naive Bayes
modelo = 'ComplementNB'

X_train, X_test, y_train, y_test = train_test_split(amostra_slug_title, df['subject'], random_state=42)

model = getattr(sklearn.naive_bayes, modelo)()
model.fit(X_train, y_train)
y_predict = model.predict(X_test)

score = accuracy_score(y_test, y_predict)

print('score --->', score)

pipe = Pipeline([('scaler', StandardScaler()), ('svc', SVC())])

pipe.fit(X_train, y_train)

dump(pipe, 'model.joblib')

print('finished')


score ---> 0.9641304347826087
finished
